In [266]:
import os

In [267]:
import warnings
warnings.filterwarnings("ignore") 

In [268]:
from dotenv import load_dotenv
load_dotenv()

True

In [269]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGSMITH_TRACING=os.getenv("LANGSMITH_TRACING_V2", "true")  # Default to "true" if not set
LANGSMITH_ENDPOINT=os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

### Load the model and test it with a simple message

In [270]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [271]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     convert_system_message_to_human=True,
# )

llm = ChatGroq(model="llama3-8b-8192")

In [272]:
message1 = [SystemMessage(content="You are a helpful assistant."),
           HumanMessage(content="I need your help")]

In [273]:
response1 = llm.invoke(message1)
response1

AIMessage(content="I'd be happy to help you. Please share what's on your mind and what you need help with. I'll do my best to assist you with your query or problem.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 25, 'total_tokens': 62, 'completion_time': 0.059432449, 'prompt_time': 0.015257287, 'queue_time': 1.0946972739999998, 'total_time': 0.074689736}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--78bce377-8069-4e7c-9252-0a66ab19bfc8-0', usage_metadata={'input_tokens': 25, 'output_tokens': 37, 'total_tokens': 62})

In [274]:
message2 = [SystemMessage(content="You are a helpful assistant. and you give concise answers."),
           HumanMessage(content="I need your help")]

In [275]:
response2 = llm.invoke(message2)
response2

AIMessage(content="I'd be happy to help. Please go ahead and tell me what's on your mind, and I'll do my best to assist you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 31, 'total_tokens': 61, 'completion_time': 0.035334073, 'prompt_time': 0.013661077, 'queue_time': 0.973936765, 'total_time': 0.04899515}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f29e2a62-6ce5-4532-af92-b2287b6a63ac-0', usage_metadata={'input_tokens': 31, 'output_tokens': 30, 'total_tokens': 61})

### Using Output Perser

In [276]:
from langchain_core.output_parsers import StrOutputParser

In [277]:
parser = StrOutputParser()
parser.invoke(response1)

"I'd be happy to help you. Please share what's on your mind and what you need help with. I'll do my best to assist you with your query or problem."

In [278]:
parser.invoke(response2)

"I'd be happy to help. Please go ahead and tell me what's on your mind, and I'll do my best to assist you."

### Chaining with LCEL
- LangChain Expression Language (LCEL) &mdash; a concise syntax for composing and chaining together components (like models and output parsers) in LangChain workflows.LCEL

In [279]:
chain = llm | parser

- The line `chain = llm | parser` creates a simple pipeline (chain) where the output of the language model (`llm`) is passed directly to the output parser (`parser`). This allows you to process a prompt through the model and automatically parse its response in a single step, making it easier to work with structured outputs in LangChain.

In [280]:
chain.invoke(message1)

"I'd be happy to help you. Please tell me what's on your mind and what you need help with. I'll do my best to provide you with accurate and helpful information or assistance."

In [281]:
chain.invoke(message2)

"I'd be happy to help. What do you need help with?"

### Prompting with PromptTemplate

In [282]:
from langchain_core.prompts import ChatPromptTemplate

In [283]:
promt_template = ChatPromptTemplate.from_messages(
    [("system", "You give concise answer and Translate the following text to : {language}"),
     ("human", "{text}")]
)
    

In [284]:
prompt = promt_template.invoke({"language": "Bengali", "text": "My name is Durjay."})

In [285]:
prompt.to_messages()

[SystemMessage(content='You give concise answer and Translate the following text to : Bengali', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Durjay.', additional_kwargs={}, response_metadata={})]

In [286]:
response = llm.invoke(prompt)

In [287]:
response.content

'আমার নাম দুর্জ্জয়। (Amāra nāma Durjay)'

### Chaining

In [288]:
chain = promt_template | llm | parser

In [289]:
chain.invoke({"language": "Bengali", "text": "My name is Durjay."})

'My answer is: আমার নাম দুর্জ্জয়।\n\n(Translation to Bengali)'

### Agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage

In [301]:
memory = MemorySaver()

In [ ]:
search = TavilySearchResults(max_results=2)

In [303]:
tools = [search]

In [304]:
agent_executor = create_react_agent(llm,tools, checkpointer=memory)

In [305]:
config = {"configurable": {"thread_id": "abc123"}}

In [306]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: attempted to call tool 'weather_search' which was not request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n    "tool_calls": [\n        {\n            "id": "pending",\n            "type": "function",\n            "function": {\n                "name": "weather_search"\n            },\n            "parameters": {\n                "query": "weather where I live"\n            }\n        }\n    ]\n}\n</tool-use>\n\nPlease note that the above JSON string is just a representation of the tool call. The actual tool call may vary depending on the specific tool and its requirements.'}}